# Specify Directory and File Information

`minian_path` is the directory containing minian, `dpath` is the directory containing the videos to be cross-registered and their corresponding minian output data (each pair of videos+output should be in a unique folder), `f_pattern` is a regular expression identifying the naming pattern of minian output folders with a regex expression (e.g. `'minian$'`, or `r'minian\.[0-9]+$'` if data is batch processed and has a timestamp), and `id_dims` should be a list containing metadata identifiers used when analyzing the individual sessions (e.g. `['session','animal']`).

In [ ]:
minian_path = "."
dpath = "./demo_movies"
f_pattern = r'minian\.[0-9]+$' 
id_dims = ['animal','session']

# Define Paramaters
`param_t_dist` defines the maximal distance between cell centroids (in pixel units) on different sessions to consider them as the same cell

In [ ]:
param_t_dist = 5
output_size = 90

# Load Modules

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import warnings
sys.path.append(minian_path)
import itertools as itt
import numpy as np
import xarray as xr
import holoviews as hv
import pandas as pd
from holoviews.operation.datashader import datashade, regrid
from dask.diagnostics import ProgressBar
from minian.cross_registration import (calculate_centroids, calculate_centroid_distance, calculate_mapping,
                                       group_by_session, resolve_mapping, fill_mapping)
from minian.motion_correction import estimate_shifts, apply_shifts
from minian.utilities import open_minian, open_minian_mf
from minian.visualization import AlignViewer
hv.notebook_extension('bokeh', width=100)
pbar = ProgressBar(minimum=2)
pbar.register()

# Allign Videos

## open datasets

In [ ]:
minian_ds = open_minian_mf(
    dpath, id_dims, pattern=f_pattern, backend='zarr')

## estimate shifts

In [ ]:
%%time
temps = minian_ds['Y'].max('frame').compute().rename('temps')
shifts = estimate_shifts(temps, max_sh=20, dim='session').compute().rename('shifts')
temps_sh = apply_shifts(temps, shifts).compute().rename('temps_shifted')
shiftds = xr.merge([temps, shifts, temps_sh])

## visualize alignment

In [ ]:
hv.output(size=output_size)
opts_im = {
    'aspect': shiftds.sizes['width'] / shiftds.sizes['height'],
    'frame_width': 500, 'cmap': 'viridis'}
hv_temps = (hv.Dataset(temps).to(hv.Image, kdims=['width', 'height'])
            .opts(**opts_im).layout('session').cols(1))
hv_temps_sh = (hv.Dataset(temps_sh).to(hv.Image, kdims=['width', 'height'])
            .opts(**opts_im).layout('session').cols(1))
display(hv_temps + hv_temps_sh)

## visualize overlap of field of view across all sessions

In [ ]:
hv.output(size=output_size)
opts_im = {
    'aspect': shiftds.sizes['width'] / shiftds.sizes['height'],
    'frame_width': 500, 'cmap': 'viridis'}
window = shiftds['temps_shifted'].isnull().sum('session')
window, temps_sh = xr.broadcast(window, shiftds['temps_shifted'])
hv_wnd = hv.Dataset(window, kdims=list(window.dims)).to(hv.Image, ['width', 'height'])
hv_temps = hv.Dataset(temps_sh, kdims=list(temps_sh.dims)).to(hv.Image, ['width', 'height'])
hv_wnd.opts(**opts_im).relabel("Window") + hv_temps.opts(**opts_im).relabel("Shifted Templates")

## apply shifts and set window

In [ ]:
A_shifted = apply_shifts(minian_ds['A'].chunk(dict(height=-1, width=-1)), shiftds['shifts'])

In [ ]:
def set_window(wnd):
    return wnd == wnd.min()
window = xr.apply_ufunc(
    set_window,
    window,
    input_core_dims=[['height', 'width']],
    output_core_dims=[['height', 'width']],
    vectorize=True)

# Calculate Centroid Distance

## calculate centroids

In [ ]:
%%time
cents = calculate_centroids(A_shifted, window)

## calculate centroid distance

In [ ]:
%%time
dist = calculate_centroid_distance(cents, index_dim=['animal'])

# Get Overlap Across Sessions

### threshold overlap based upon centroid distance and generate overlap mappings

In [ ]:
dist_ft = dist[dist['variable', 'distance'] < param_t_dist].copy()
dist_ft = group_by_session(dist_ft)

In [ ]:
%%time
mappings = calculate_mapping(dist_ft)
mappings_meta = resolve_mapping(mappings)
mappings_meta_fill = fill_mapping(mappings_meta, cents)
mappings_meta_fill.head()

### save results

In [ ]:
mappings_meta_fill.to_pickle(os.path.join(dpath, "mappings.pkl"))
cents.to_pickle(os.path.join(dpath, "cents.pkl"))
shiftds.to_netcdf(os.path.join(dpath, "shiftds.nc"))

# View Overlap Across Any 3 Sessions

In [ ]:
mappings_meta_fill = pd.read_pickle(os.path.join(dpath, "mappings.pkl"))
cents = pd.read_pickle(os.path.join(dpath, "cents.pkl"))
shiftds = xr.open_dataset(os.path.join(dpath, 'shiftds.nc'))

In [ ]:
hv.output(size=output_size)
alnviewer = AlignViewer(minian_ds, cents, mappings_meta_fill, shiftds)
alnviewer.show()